In [1]:
import os
import sys

# 获取项目根目录的绝对路径
current_dir = os.path.dirname(os.path.abspath(''))

# 确保项目根目录在系统路径中
if current_dir not in sys.path:
    sys.path.append(current_dir)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from student.output import DeepResNet
from data.distill_dataset import DistillDataManager


# 加载数据

In [3]:
radius_save_path = "../data/radius_matrix.pth"

if os.path.exists(radius_save_path):
    radius_matrix = torch.load(radius_save_path, weights_only=False)
else:
    radius_matrix = torch.rand(10, 10) * 10
    torch.save(radius_matrix, radius_save_path)

print(radius_matrix)

tensor([[4.2537, 7.2491, 6.1717, 9.2729, 4.9853, 9.8473, 3.1820, 9.3614, 8.1104,
         9.0288],
        [2.3149, 8.7649, 5.0795, 8.0234, 4.8227, 8.7122, 0.7336, 2.7354, 7.6845,
         6.5359],
        [6.5778, 9.8060, 0.8749, 7.8437, 6.0056, 7.5152, 8.9349, 6.0827, 4.3046,
         7.7731],
        [0.3096, 1.4307, 0.4254, 2.0687, 3.2475, 0.8592, 9.9883, 3.1064, 7.1883,
         2.8260],
        [6.0670, 4.3900, 8.1416, 5.1876, 1.6904, 2.8222, 9.9509, 8.0686, 6.5593,
         5.9664],
        [9.7981, 5.3376, 9.5862, 0.3684, 5.5181, 7.1683, 3.2136, 5.2784, 1.8340,
         9.4031],
        [2.1909, 0.4959, 9.4064, 1.8535, 2.7756, 7.6035, 9.4730, 6.0837, 7.2479,
         1.7767],
        [2.4067, 8.6760, 5.0395, 6.0531, 6.9528, 0.0580, 0.6878, 2.1934, 4.4764,
         4.8344],
        [4.9628, 6.8850, 0.1238, 5.7276, 2.3660, 0.6239, 9.2926, 6.6920, 3.6652,
         1.5679],
        [2.8365, 5.7207, 1.2948, 2.9172, 4.4617, 8.5791, 4.1536, 8.3898, 6.0025,
         7.9040]])


In [4]:
data_manager = DistillDataManager(radius_matrix)

train_dataset = data_manager.get_train_dataset(mode='output')
print("train_dataset loaded.")
test_dataset = data_manager.get_test_dataset(mode='output')
print("test_dataset loaded.")

print(train_dataset)
print(test_dataset)

train_dataset loaded.


# 训练模型

In [ ]:
model = DeepResNet(
    input_dim=10,
    output_dim=10,
    hidden_dims=[512, 1024, 2048, 1024, 512]
)

model = data_manager.load_model(model, mode='output')

num_epochs = 100000
learning_rate = 1e-3
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [ ]:
train_loss = []
best_loss = float('inf')
best_state_dict = None
best_epochs = 0
total_steps = num_epochs * len(train_loader)
progress_bar = tqdm(total=total_steps, desc="Training Progress")
last_epoch_loss = 0

with torch.enable_grad():
    for epoch in range(num_epochs):
        epoch_loss = 0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            progress_bar.update(1)

            progress_bar.set_description(
                f"Epoch {epoch+1}, Loss: {loss.item():.4f}" 
                + f", LastEpochLoss: {last_epoch_loss:.4f}"
            )
            
        last_epoch_loss = epoch_loss / len(train_loader)
        train_loss.append(last_epoch_loss)

        if last_epoch_loss < best_loss:
            best_loss = last_epoch_loss
            best_state_dict = model.state_dict()
            best_epochs = epoch

progress_bar.close()
print(f"Best Epochs: {best_epochs}; Best Loss: {best_loss:.4f}")

In [ ]:
if best_state_dict is not None:
    model.load_state_dict(best_state_dict)

data_manager.save_model(model, mode='output')

# 测试模型

In [ ]:
model = DeepResNet(
    input_dim=10,
    output_dim=10,
    hidden_dims=[512, 1024, 2048, 1024, 512]
)

model = data_manager.load_model(model, mode='output')

In [ ]:
model.eval()

test_loss = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        all_preds.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.6f}")

In [ ]:
# 将预测和标签转换为numpy数组以便绘图
all_preds = np.concatenate(all_preds, axis=0)  # [n_samples, n_ports]
all_labels = np.concatenate(all_labels, axis=0)  # [n_samples, n_ports]

# 创建图形
plt.figure(figsize=(15, 5))

# 1. 样本预测对比图
plt.subplot(131)
sample_idx = 0  # 可以改变这个索引来查看不同的样本
plt.plot(all_preds[sample_idx], 'b-o', label='预测值')
plt.plot(all_labels[sample_idx], 'r-o', label='真实值')
plt.title(f'样本 {sample_idx} 的预测结果')
plt.xlabel('端口')
plt.ylabel('强度')
plt.legend()
plt.grid(True)

# 2. 预测值与真实值的散点图
plt.subplot(132)
plt.scatter(all_labels.flatten(), all_preds.flatten(), alpha=0.5)
plt.plot([0, 1], [0, 1], 'r--')  # 理想的对角线
plt.title('预测值 vs 真实值')
plt.xlabel('真实值')
plt.ylabel('预测值')
plt.grid(True)

# 3. 每个端口的误差箱型图
plt.subplot(133)
errors = all_preds - all_labels
plt.boxplot(errors)
plt.title('各端口预测误差分布')
plt.xlabel('端口')
plt.ylabel('误差')
plt.grid(True)

plt.tight_layout()
plt.show()

# 计算统计指标
mse = np.mean((all_preds - all_labels) ** 2)
mae = np.mean(np.abs(all_preds - all_labels))
r2 = 1 - np.sum((all_preds - all_labels) ** 2) / np.sum((all_labels - np.mean(all_labels)) ** 2)

print("整体性能指标:")
print(f"均方误差 (MSE): {mse:.6f}")
print(f"平均绝对误差 (MAE): {mae:.6f}")
print(f"R² 分数: {r2:.6f}")

# 计算每个端口的性能指标
print("\n各端口性能指标:")
for port in range(all_preds.shape[1]):
    port_mse = np.mean((all_preds[:, port] - all_labels[:, port]) ** 2)
    port_mae = np.mean(np.abs(all_preds[:, port] - all_labels[:, port]))
    port_r2 = 1 - np.sum((all_preds[:, port] - all_labels[:, port]) ** 2) / \
              np.sum((all_labels[:, port] - np.mean(all_labels[:, port])) ** 2)
    print(f"\n端口 {port}:")
    print(f"  MSE: {port_mse:.6f}")
    print(f"  MAE: {port_mae:.6f}")
    print(f"  R²: {port_r2:.6f}")